In [1]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

In [2]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)
P_LED = LED_ring.P
D_LED = LED_ring.D

## get LED intensity ##
A = 0.4
B = 0.5
# LED_ring.set_sine_intensity(A,B)
LED_ring.set_sine_intensity(A,B)


In [3]:
Intrinsics = mi.Matrix3f([[422.3949, 0,313.5413],[0,422.4885,233.1770],[0,0,1]])
Z_camera = -44
image_res = (640,480)
cam_conner1 = dr.inverse(Intrinsics)@mi.Point3f(0,0,1)*-Z_camera
cam_conner2 = dr.inverse(Intrinsics)@mi.Point3f(image_res[0]-1,image_res[1]-1,1)*-Z_camera


# ### camera origin          ###
cam_origin = mi.Point3f(0,0,Z_camera)
cam_range = ((cam_conner1[0,0],cam_conner1[1,0]),(cam_conner2[0,0],cam_conner2[1,0]))
cam_fov= (cam_conner2[0,0]-cam_conner1[0,0],cam_conner2[1,0]-cam_conner1[1,0])
from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_range)

In [ ]:
import time
from utils import iristac

start_time = time.time()
scene = mi.load_file('../scenes/iristac_ply.xml')

sensor = iristac.sensor(cam,LED_ring)
sensor.Z0_scene = scene
sensor.compute_R()
I_RGB = sensor.get_tactile_img()


end_time = time.time()
print("Time used: {:.10f} seconds".format(end_time - start_time))

In [ ]:
import matplotlib.pyplot as plt

plt.axis('off')
plt.imshow(I_RGB)
plt.show()

In [6]:
bmp = mi.Bitmap(I_RGB)
bmp = bmp.convert(mi.Bitmap.PixelFormat.RGB, mi.Struct.Type.UInt8, False)
bmp.write('../scenes/img/normal-10-10.png')